<a href="https://colab.research.google.com/github/fakhia/gene-to-protein-analysis/blob/main/Codon_Usage_Profiling_of_Human_TRPV6_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# trpv6_codon_analysis.py
# Requirements: biopython, pandas, numpy, matplotlib, scipy
# pip install biopython pandas numpy matplotlib scipy

from Bio import Entrez, SeqIO
from collections import Counter, defaultdict
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log
from scipy.stats import pearsonr, spearmanr

Entrez.email = "your_email@example.com"  # set your email


##### Utilities #####
CODON_TABLE = {
    'TTT':'F','TTC':'F','TTA':'L','TTG':'L','CTT':'L','CTC':'L','CTA':'L','CTG':'L',
    'ATT':'I','ATC':'I','ATA':'I','ATG':'M','GTT':'V','GTC':'V','GTA':'V','GTG':'V',
    'TCT':'S','TCC':'S','TCA':'S','TCG':'S','CCT':'P','CCC':'P','CCA':'P','CCG':'P',
    'ACT':'T','ACC':'T','ACA':'T','ACG':'T','GCT':'A','GCC':'A','GCA':'A','GCG':'A',
    'TAT':'Y','TAC':'Y','TAA':'*','TAG':'*','CAT':'H','CAC':'H','CAA':'Q','CAG':'Q',
    'AAT':'N','AAC':'N','AAA':'K','AAG':'K','GAT':'D','GAC':'D','GAA':'E','GAG':'E',
    'TGT':'C','TGC':'C','TGA':'*','TGG':'W','CGT':'R','CGC':'R','CGA':'R','CGG':'R',
    'AGT':'S','AGC':'S','AGA':'R','AGG':'R','GGT':'G','GGC':'G','GGA':'G','GGG':'G'
}

SYNONYMS = defaultdict(list)
for codon, aa in CODON_TABLE.items():
    if aa != '*':
        SYNONYMS[aa].append(codon)

def fetch_cds_by_refseq(refseq_id):
    handle = Entrez.efetch(db="nucleotide", id=refseq_id, rettype="fasta", retmode="text")
    rec = SeqIO.read(handle, "fasta")
    handle.close()
    seq = str(rec.seq).upper().replace("\n","").replace(" ","")
    return seq

def fetch_cds_by_gene_species(gene_name, species, maxrecords=5):
    # returns first CDS sequence fasta record matching gene and species
    query = f"{gene_name}[Gene] AND {species}[Organism] AND cds[Feature]"
    handle = Entrez.esearch(db="nucleotide", term=query, retmax=maxrecords)
    res = Entrez.read(handle)
    handle.close()
    ids = res.get('IdList', [])
    if not ids:
        return None
    # fetch first record as GenBank to parse CDS if present
    for id_ in ids:
        handle = Entrez.efetch(db="nucleotide", id=id_, rettype="gb", retmode="text")
        try:
            rec = SeqIO.read(handle, "genbank")
        except Exception:
            handle.close()
            continue
        handle.close()
        # look for CDS feature
        for f in rec.features:
            if f.type == "CDS":
                seq = f.extract(rec.seq).upper()
                return str(seq)
    return None


def codon_counts(dna_seq):
    dna_seq = dna_seq.upper()
    codons = [dna_seq[i:i+3] for i in range(0, len(dna_seq)-2, 3) if len(dna_seq[i:i+3])==3]
    return Counter(codons), len(codons)

def codon_freq_table(counter, total_codons):
    freqs = {codon: counter.get(codon,0)/total_codons for codon in sorted(CODON_TABLE.keys())}
    return freqs

def rscu(counter):
    # Relative Synonymous Codon Usage
    # RSCU = observed_count / (expected_count assuming equal usage among synonymous codons)
    rscu_vals = {}
    for aa, codons in SYNONYMS.items():
        total = sum(counter.get(c,0) for c in codons)
        n = len(codons)
        if total == 0:
            for c in codons:
                rscu_vals[c]=0.0
        else:
            for c in codons:
                rscu_vals[c] = (counter.get(c,0) * n) / total
    return rscu_vals

def gc3(dna_seq):
    dna_seq = dna_seq.upper()
    third_positions = [dna_seq[i+2] for i in range(0, len(dna_seq)-2, 3) if len(dna_seq[i:i+3])==3]
    gc = sum(1 for b in third_positions if b in ('G','C'))/len(third_positions)
    return gc

def effective_number_of_codons(counter):
    # A simple approximation (Wright's ENC is more complex). We'll provide a rough measure.
    # For publication-quality ENC, use a library. Here we use a proxy: sum of homozygosity per aa.
    F_vals = []
    for aa, codons in SYNONYMS.items():
        counts = np.array([counter.get(c,0) for c in codons], dtype=float)
        n = counts.sum()
        if n <= 1:
            continue
        freqs = counts / n
        F = (freqs**2).sum()
        if np.isnan(F):
            continue
        F_vals.append(F)
    if not F_vals:
        return np.nan
    # transform to a number in ENC-like scale (not exact)
    ENC_proxy = 2 + 9/(np.mean(F_vals)+1e-9)  # heuristic
    return float(ENC_proxy)

def compute_cai(dna_seq, reference_usage):
    """
    Compute CAI using reference codon usage (dict of codon->freq)
    reference_usage should be freq for each codon (not counts) normalized per amino acid
    CAI = geometric mean of relative adaptiveness (w) values across codons
    w(codon) = f_codon / max_f_for_that_aa_in_reference
    """
    dna_seq = dna_seq.upper()
    codons = [dna_seq[i:i+3] for i in range(0, len(dna_seq)-2, 3) if len(dna_seq[i:i+3])==3]
    w_values = []
    for c in codons:
        aa = CODON_TABLE.get(c,'X')
        if aa == '*' or aa == 'X':  # skip stops or unknown
            continue
        # get all codons for that aa
        aa_codons = SYNONYMS[aa]
        max_f = max(reference_usage.get(k,1e-9) for k in aa_codons)
        f_c = reference_usage.get(c, 0.0)
        w = (f_c / max_f) if max_f>0 else 0.0
        w_values.append(w if w>0 else 1e-6)
    # geometric mean
    if not w_values:
        return np.nan
    log_mean = sum(np.log(w_values)) / len(w_values)
    return float(np.exp(log_mean))

##### Main pipeline #####

def task1_compare_human_mouse(human_ref="NM_018646.6", mouse_gene="Trpv6"):
    # Fetch human by refseq
    human_seq = fetch_cds_by_refseq(human_ref)
    mouse_seq = fetch_cds_by_gene_species(mouse_gene, "Mus musculus")
    if mouse_seq is None:
        raise ValueError("Could not fetch mouse Trpv6 CDS automatically. Provide RefSeq or FASTA manually.")
    # counts
    human_counts, human_total = codon_counts(human_seq)
    mouse_counts, mouse_total = codon_counts(mouse_seq)
    human_freq = codon_freq_table(human_counts, human_total)
    mouse_freq = codon_freq_table(mouse_counts, mouse_total)
    human_rscu = rscu(human_counts)
    mouse_rscu = rscu(mouse_counts)

    # Save tables
    df = pd.DataFrame({
        "codon": sorted(CODON_TABLE.keys()),
        "aa": [CODON_TABLE[c] for c in sorted(CODON_TABLE.keys())],
        "human_count": [human_counts.get(c,0) for c in sorted(CODON_TABLE.keys())],
        "human_freq": [human_freq[c] for c in sorted(CODON_TABLE.keys())],
        "human_rscu": [human_rscu.get(c,0) for c in sorted(CODON_TABLE.keys())],
        "mouse_count": [mouse_counts.get(c,0) for c in sorted(CODON_TABLE.keys())],
        "mouse_freq": [mouse_freq[c] for c in sorted(CODON_TABLE.keys())],
        "mouse_rscu": [mouse_rscu.get(c,0) for c in sorted(CODON_TABLE.keys())],
    })
    df.to_csv("trpv6_codon_compare_human_mouse.csv", index=False)

    # Plots
    codons = df['codon'].tolist()
    plt.figure(figsize=(20,6))
    plt.plot(codons, df['human_freq'], label='Human TRPV6', marker='o')
    plt.plot(codons, df['mouse_freq'], label='Mouse Trpv6', marker='s')
    plt.xticks(rotation=90)
    plt.ylabel("Relative frequency")
    plt.title("Codon frequency: Human vs Mouse TRPV6")
    plt.legend()
    plt.tight_layout()
    plt.savefig("codon_freq_compare.png", dpi=300)
    plt.close()

    plt.figure(figsize=(20,6))
    plt.plot(codons, df['human_rscu'], label='Human RSCU', marker='o')
    plt.plot(codons, df['mouse_rscu'], label='Mouse RSCU', marker='s')
    plt.xticks(rotation=90)
    plt.ylabel("RSCU")
    plt.title("RSCU: Human vs Mouse TRPV6")
    plt.legend()
    plt.tight_layout()
    plt.savefig("rscu_compare.png", dpi=300)
    plt.close()

    print("Saved: trpv6_codon_compare_human_mouse.csv, codon_freq_compare.png, rscu_compare.png")
    return human_seq, mouse_seq, df

def task2_metrics_and_correlation(dna_seq, expression_values=None, expression_label=None, reference_usage=None):
    """
    dna_seq: CDS for a gene (string)
    expression_values: a dict or pd.Series mapping tissue/sample -> expression value (optional)
    reference_usage: dict codon->freq used to compute CAI. If None, we can use the gene itself as trivial ref (not recommended).
    """
    counts, total = codon_counts(dna_seq)
    freqs = codon_freq_table(counts, total)
    rscu_vals = rscu(counts)
    gc3_val = gc3(dna_seq)
    ENC_proxy = effective_number_of_codons(counts)
    if reference_usage is None:
        # WARNING: better to compute reference from many highly expressed genes; here we fallback to using freqs
        reference_usage = freqs
    cai_val = compute_cai(dna_seq, reference_usage)

    metrics = {
        "gene_length_bp": len(dna_seq),
        "codon_count": total,
        "GC3": gc3_val,
        "ENC_proxy": ENC_proxy,
        "CAI": cai_val
    }
    # Save detailed codon table
    codon_df = pd.DataFrame({
        "codon": sorted(CODON_TABLE.keys()),
        "aa": [CODON_TABLE[c] for c in sorted(CODON_TABLE.keys())],
        "count": [counts.get(c,0) for c in sorted(CODON_TABLE.keys())],
        "freq": [freqs[c] for c in sorted(CODON_TABLE.keys())],
        "rscu": [rscu_vals.get(c,0) for c in sorted(CODON_TABLE.keys())]
    })
    codon_df.to_csv("trpv6_codon_metrics_table.csv", index=False)
    pd.Series(metrics).to_csv("trpv6_codon_summary_metrics.csv")

    print("Saved: trpv6_codon_metrics_table.csv, trpv6_codon_summary_metrics.csv")
    print("Metrics:", metrics)

    # If expression provided, compute correlation (single gene: we can correlate metric vs expression across samples if you have multiple genes;
    # for single gene across tissues you can correlate CAI vs expression across tissues only if CAI varies (it doesn't per gene).
    # More typical: compute metrics across a gene set and correlate with expression across the same gene set.)
    if expression_values is not None:
        # If expression_values is a dict mapping tissue->expression for same gene, we can compare tissues vs expression but codon metrics are constant per gene.
        # Instead, this function assumes you provide a small dataframe of multiple genes' metrics and expression to correlate.
        expr = pd.Series(expression_values, name='expression')
        # For single gene case, we can just plot expression across tissues; for correlation you need metrics for many genes.
        expr.to_csv("trpv6_expression_values.csv")
        plt.figure(figsize=(8,4))
        expr.plot(kind='bar')
        plt.title(expression_label or "Expression values")
        plt.ylabel("Expression (user-provided units)")
        plt.tight_layout()
        plt.savefig("trpv6_expression_plot.png", dpi=300)
        plt.close()
        print("Saved expression plot: trpv6_expression_plot.png")
    return metrics, codon_df

##### Example run #####
if __name__ == "__main__":
    # Task 1
    human_ref = "NM_018646.6"
    print("Task 1: comparing human (RefSeq {}) with mouse".format(human_ref))
    human_seq, mouse_seq, compare_df = task1_compare_human_mouse(human_ref=human_ref, mouse_gene="Trpv6")

    # Task 2: compute metrics for human TRPV6
    print("\nTask 2: computing codon metrics for human TRPV6")
    # To compute CAI properly, supply reference_usage computed from many highly expressed human genes (not included here)
    metrics, codon_df = task2_metrics_and_correlation(human_seq, reference_usage=None)

    print("\nDone. Check output files in working directory.")


Task 1: comparing human (RefSeq NM_018646.6) with mouse
Saved: trpv6_codon_compare_human_mouse.csv, codon_freq_compare.png, rscu_compare.png

Task 2: computing codon metrics for human TRPV6
Saved: trpv6_codon_metrics_table.csv, trpv6_codon_summary_metrics.csv
Metrics: {'gene_length_bp': 2906, 'codon_count': 968, 'GC3': 0.4359504132231405, 'ENC_proxy': 21.700658666601004, 'CAI': 0.7522576439461429}

Done. Check output files in working directory.
